In [6]:
import warnings
#warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import mne
from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_gdf
from mne.decoding import CSP
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from EEGModels import EEGNet
from sklearn.metrics import confusion_matrix


'''
Parameters:
'''
tmin = 0
tmax = 10

In [7]:
object_names = ['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10']
# object_names = ['P01', 'P02', 'P03', 'P04']
files = []
for name in object_names:
    for i in range(15):
        files.append(read_raw_gdf(f'..\\{name}\\{name} Run {i+1}.gdf', stim_channel=None, eog=[61,62,63]))

Extracting EDF parameters from D:\EEGTest\Attempted arm and hand movements in persons with spinal cord injury\P01\P01 Run 1.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEGTest\Attempted arm and hand movements in persons with spinal cord injury\P01\P01 Run 2.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEGTest\Attempted arm and hand movements in persons with spinal cord injury\P01\P01 Run 3.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEGTest\Attempted arm and hand movements in persons with spinal cord injury\P01\P01 Run 4.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\EEGTest\Attempted arm and hand movements in persons with spinal cord injury\P01\P01 Run 5.gdf...
GDF file dete

C:\Users\kings\Anaconda3\envs\mne\lib\site-packages\mne\io\edf\edf.py:996: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
C:\Users\kings\Anaconda3\envs\mne\lib\site-packages\mne\io\edf\edf.py:996: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
C:\Users\kings\Anaconda3\envs\mne\lib\site-packages\mne\io\edf\edf.py:996: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
C:\Users\kings\Anaconda3\envs\mne\lib\site-packages\mne\io\edf\edf.py:996: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead

In [8]:
print(len(files))

150


In [9]:
eventDescription_offline_paradigm = {
    '768': "trial start",
    '785': "beep",
    '786': "fixation cross",
    '776': "supinationclass cue",
    '777': "pronationclass cue",
    '779': "hand openclass cue",
    '925': "palmar graspclass cue",
    '926': "ateral graspclass cue",
}
# narrow it down to two
eventDescription_offline_paradigm = {
    '785': "beep",
    '777': "pronationclass cue",
    '779': "hand openclass cue",
}
nb_class = len(eventDescription_offline_paradigm)

In [10]:
# wave frequency
iter_freqs = [
    ('Theta', 4, 7),
    ('Alpha', 8, 12),
    ('Beta', 13, 25),
    ('Gamma', 30, 45)
]


In [23]:
# events = [number of events, position, event code, channel, duration]
# file_P01_Run3._raw_extras[0]['events']
event_id = {}
epochs_list = []
for file in files:
    file.load_data()
    file.filter(iter_freqs[2][1], iter_freqs[3][2], l_trans_bandwidth=1, h_trans_bandwidth=1, verbose=0)
    event, _ = mne.events_from_annotations(file)


    # build event id and filter 1-7 id

    contains_both = 0
    for i in _: #handle event_id
        if i not in eventDescription_offline_paradigm:
            continue
        contains_both += 1
        event_id[eventDescription_offline_paradigm[i]] = _[i]


    # print(f'event id: {event_id}')
    if contains_both == len(eventDescription_offline_paradigm):
        epochs = mne.Epochs(file, event, event_id, tmin=-0., tmax=1, baseline=None, event_repeated = 'merge', preload=True)
        epochs.subtract_evoked()
        # epochs.apply_hilbert(envelope=True)
        epochs_list.append(epochs)

result_epochs = mne.concatenate_epochs(epochs_list)

Used Annotations descriptions: ['33536', '768', '785']
Used Annotations descriptions: ['33536', '33554', '768', '785', '786']
Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']
Not setting metadata
Not setting metadata
56 matching events found
No baseline correction applied
0 projection items activated
Loading data for 56 events and 257 original time points ...
0 bad epochs dropped
Subtracting Evoked from Epochs
    The following channels are not included in the subtraction: eog-m, eog-r, eog-l
[done]
Used Annotations descriptions: ['33536', '33544', '33545', '33547', '33554', '33693', '33694', '768', '776', '777', '779', '785', '786', '925', '926']
Not setting metadata
Not setting metadata
56 matching events found
No baseline correction applied
0 projection items activated
Loading data for 56 events and 257 original time points ...
0 bad epochs dropped
Subtracting Evoked from Epochs
   

In [25]:
print(len(set(event_id)))
#9, 10, 0
print(result_epochs.events)

3
[[   2560       0      12]
 [   3072       0      10]
 [   4409       0      12]
 ...
 [6816486       0      12]
 [6816998       0      11]
 [6818468       0      12]]


In [26]:
# 0: not supinationclass or pronationclass cue
# 0: not supinationclass //corrected
labels = result_epochs.events[:,-1]
# labels = np.array(list(map(lambda x:  0 if (not x == 9) else x, labels)))
print(labels)

# format: trials, channels, samples
X = result_epochs.get_data() * 1000
y = labels

kernels, chans, samples = 1, X.shape[1], X.shape[2]

[12 10 12 ... 12 11 12]


In [27]:
X.shape

(5040, 64, 257)

In [28]:
train_ratio = int(X.shape[0] * 0.7)
validate_ratio = int(X.shape[0] * 0.8)
# Spliting dataset
# 70% Training, 10% Validate, 20% Testing
X_train = X[0:train_ratio,]
Y_train = y[0:train_ratio]
X_validate = X[train_ratio:validate_ratio,]
Y_validate = y[train_ratio:validate_ratio]
X_test = X[validate_ratio:,]
Y_test = y[validate_ratio:]

print(Y_train)

[12 10 12 ... 11 12 12]


In [29]:
# Label Encode it first
encoder = LabelEncoder()
encoder.fit(Y_train)
Y_train = encoder.transform(Y_train)
Y_validate = encoder.transform(Y_validate)
Y_test = encoder.transform(Y_test)

# Convert label to one-hot encodings
Y_train = np_utils.to_categorical(Y_train)
Y_validate = np_utils.to_categorical(Y_validate)
Y_test = np_utils.to_categorical(Y_test)

#convert to (trials, kernels, channels, samples) format.
# contains 64 channels and 257 time-points. Set the number of kernels to 1.
X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
X_validate = X_validate.reshape(X_validate.shape[0], kernels, chans, samples)
X_test = X_test.reshape(X_test.shape[0], kernels, chans, samples)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (3528, 1, 64, 257)
3528 train samples
1008 test samples


In [30]:
# testing gpu
from tensorflow.python.client import device_lib
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())
gpus = tf.config.experimental.list_physical_devices('GPU')
for physical_device in gpus:
    tf.config.experimental.set_memory_growth(physical_device, True)
# change the memory fraction as you want...import tensorflow as tf
# if gpus:
#     # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
#     try:
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Virtual devices must be set before GPUs have been initialized
#         print(e)
print(gpus)

['/device:CPU:0', '/device:XLA_CPU:0', '/device:GPU:0', '/device:XLA_GPU:0']
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [31]:
# configure the EEGNet-8,2,16 model with kernel length of 257 samples (other
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = nb_class, Chans = chans, Samples = samples,
               dropoutRate = 0.5, kernLength = 128, F1 = 8, D = 2, F2 = 16)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1, 64, 257)]      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 64, 257)        1024      
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 64, 257)        32        
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 16, 1, 257)        1024      
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 1, 257)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 16, 1, 257)        0         
_________________________________________________________________
average_pooling2d_2 (Average (None, 16, 1, 64)        

In [32]:
# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='./checkpoint.h5', verbose=1,
                               save_best_only=True)


In [33]:
# add callback
losses = []
accs = []
class LossAndAccRecord(Callback):
    def on_batch_end(self, batch, logs=None):
        losses.append(logs["loss"])
        accs.append(logs["accuracy"])


    def on_epoch_end(self, epoch, logs=None):
        pass


In [34]:
# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
# unused
class_weights = {0:3, 1:1, 2:1}
with tf.device('/device:GPU:0'):
    fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 30,
                        verbose = 1, validation_data=(X_validate, Y_validate),
                        class_weight=class_weights,
                        callbacks=[checkpointer, LossAndAccRecord()])

Epoch 1/30
221/221 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1429 

KeyboardInterrupt: 

In [ ]:
# load optimal weights
model.load_weights('./checkpoint.h5')

In [ ]:
###############################################################################
# make prediction on test set.
###############################################################################

probs       = model.predict(X_test)
print(probs)
preds       = probs.argmax(axis = -1)
acc         = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %.7f " % (acc))

In [ ]:
confusion_matrix(Y_test.argmax(axis=-1), preds)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2)
ax[0].plot(losses)
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epochs')
ax[1].plot(accs)
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epochs')
plt.show()